# PyCaret Fugue Integration

[Fugue](https://github.com/fugue-project/fugue) is a low-code unified interface for different computing frameworks such as Spark, Dask and Pandas. PyCaret is using Fugue to support distributed computing scenarios.

## Hello World

Let's start with the most standard example, the code is exactly the same as the local version, there is no magic.

In [1]:
from pycaret.datasets import get_data
from pycaret.classification import *

setup(data=get_data("juice"), target = 'Purchase', n_jobs=1)

test_models = models().index.tolist()[:5]

,Description,Value
0,session_id,7971
1,Target,Purchase
2,Target Type,Binary
3,Label Encoded,"CH: 0, MM: 1"
4,Original Data,"(1070, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


`compare_model` is also exactly the same if you don't want to use a distributed system

In [2]:
compare_models(include=test_models, n_select=2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8289,0.9038,0.7700,0.8026,0.7831,0.6422,0.6455,0.0950
dt,Decision Tree Classifier,0.7795,0.7736,0.7333,0.7292,0.7279,0.5432,0.5467,0.0100
nb,Naive Bayes,0.7714,0.8448,0.7667,0.7024,0.7305,0.5336,0.5380,0.0080
knn,K Neighbors Classifier,0.7313,0.7746,0.6500,0.6726,0.6603,0.4383,0.4392,0.0110
svm,SVM - Linear Kernel,0.5576,0.0000,0.5367,0.3176,0.3734,0.1122,0.1143,0.0100


[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=7971, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=7971, splitter='best')]

Now let's make it distributed, as a toy case, on dask. The only thing changed is an additional parameter `parallel_backend`

In [3]:
from pycaret.parallel import FugueBackend

compare_models(include=test_models, n_select=2, parallel=FugueBackend("dask"))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=7971, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=7971, splitter='best')]

In order to use Spark as the execution engine, you must have access to a Spark cluster, and you must have a `SparkSession`, let's initialize a local Spark session

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Now just change `parallel_backend` to this session object, you make it run on Spark. You must understand this is a toy case. In the real situation, you need to have a SparkSession pointing to a real Spark cluster to enjoy the power of Spark

In [6]:
compare_models(include=test_models, n_select=2, parallel=FugueBackend(spark))

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=5258, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09)]

In the end, you can `pull` to get the metrics table

In [7]:
pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8289,0.9004,0.7543,0.8036,0.7741,0.6372,0.6419,0.100
nb,Naive Bayes,0.7660,0.8351,0.7720,0.6775,0.7205,0.5210,0.5258,0.016
dt,Decision Tree Classifier,0.7608,0.7520,0.6668,0.7115,0.6872,0.4942,0.4957,0.015
knn,K Neighbors Classifier,0.7125,0.7569,0.5589,0.6665,0.6046,0.3825,0.3881,0.018
svm,SVM - Linear Kernel,0.5657,0.0000,0.3677,0.2429,0.2429,0.0572,0.0601,0.018


## A more practical case

The above example is a pure toy, to make things work perfectly in a distributed system you must be careful about a few things

### Use a lambda instead of a dataframe in setup

If you directly provide a dataframe in `setup`, this dataset will need to be sent to all worker nodes. If the dataframe is 1G, you have 100 workers, then it is possible your dirver machine will need to send out up to 100G data (depending on specific framework's implementation), then this data transfer becomes a bottleneck itself. Instead, if you provide a lambda function, it doesn't change the local compute scenario, but the driver will only send the function reference to workers, and each worker will be responsible to load the data by themselves, so there is no heavy traffic on the driver side.

### Be deterministic

You should always use `session_id` to make the distributed compute deterministic, otherwise, for the exactly same logic you could get drastically different selection for each run.

### Set n_jobs

It is important to be explicit on n_jobs when you want to run something distributedly, so it will not overuse the local/remote resources. This can also avoid resrouce contention, and make the compute faster.

In [9]:
setup(data=lambda: get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=1);

,Description,Value
0,session_id,0
1,Target,Purchase
2,Target Type,Binary
3,Label Encoded,"CH: 0, MM: 1"
4,Original Data,"(1070, 19)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


### Set the appropriate batch_size

`batch_size` parameter helps adjust between load balence and overhead. For each batch, setup will be called only once. So

| Choice |Load Balance|Overhead|Best Scenario|
|---|---|---|---|
|Smaller batch size|Better|Worse|`training time >> data loading time` or `models ~= workers`|
|Larger batch size|Worse|Better|`training time << data loading time` or `models >> workers`|

The default value is set to `1`, meaning we want the best load balance.

### Display progress

In development, you can enable visual effect by `display_remote=True`, but meanwhile you must also enable [Fugue Callback](https://fugue-tutorials.readthedocs.io/tutorials/advanced/rpc.html) so that the driver can monitor worker progress. But it is recommended to turn off display in production.

In [10]:
fconf = {
    "fugue.rpc.server": "fugue.rpc.flask.FlaskRPCServer",  # keep this value
    "fugue.rpc.flask_server.host": "0.0.0.0",  # the driver ip address workers can access
    "fugue.rpc.flask_server.port": "3333",  # the open port on the dirver
    "fugue.rpc.flask_server.timeout": "2 sec",  # the timeout for worker to talk to driver
}

be = FugueBackend("dask", fconf, display_remote=True, batch_size=3, top_only=False)
compare_models(n_select=2, parallel=be)

IntProgress(value=0, description='Processing: ', max=17)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8329,0.8950,0.7658,0.7945,0.7746,0.6425,0.6480,0.012
ridge,Ridge Classifier,0.8275,0.0000,0.7479,0.7971,0.7654,0.6299,0.6366,0.019
lr,Logistic Regression,0.8248,0.8965,0.7195,0.8089,0.7540,0.6195,0.6284,0.110
gbc,Gradient Boosting Classifier,0.8208,0.8869,0.7509,0.7792,0.7613,0.6182,0.6219,0.115
rf,Random Forest Classifier,0.8074,0.8779,0.7443,0.7534,0.7447,0.5907,0.5947,0.170
ada,Ada Boost Classifier,0.8035,0.8680,0.7014,0.7672,0.7292,0.5757,0.5804,0.091
lightgbm,Light Gradient Boosting Machine,0.8034,0.8799,0.7405,0.7446,0.7394,0.5821,0.5850,0.049
mlp,MLP Classifier,0.8021,0.8876,0.7127,0.7703,0.7230,0.5724,0.5868,0.788
et,Extra Trees Classifier,0.7834,0.8514,0.7053,0.7250,0.7078,0.5369,0.5439,0.144
dt,Decision Tree Classifier,0.7751,0.7649,0.6977,0.7064,0.6998,0.5204,0.5226,0.009


[LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                            solver='svd', store_covariance=False, tol=0.0001),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, normalize=False, random_state=0, solver='auto',
                 tol=0.001)]

### A note on Spark settings

It is highly recommended to have only 1 worker on each Spark executor, so the worker can fully utilize all cpus (set `spark.task.cpus`). Also when you do this you should explicitly set `n_jobs` in `setup` to the number of cpus of each executor.

```python
executor_cores = 4

spark = SparkSession.builder.config("spark.task.cpus", executor_cores).config("spark.executor.cores", executor_cores).getOrCreate()

setup(data=get_data("juice", verbose=False, profile=False), target = 'Purchase', session_id=0, n_jobs=executor_cores)

compare_models(n_select=2, parallel_backend=spark)
```

### A note on Dask

Dask has fake distributed modes such as the default (multi-thread) and multi-process modes. The default mode will just work fine (but they are actually running sequentially), and multi-process doesn't work for PyCaret for now because it messes up with PyCaret's global variables. On the other hand, any Spark execution mode will just work fine.